In [3]:
import json
import os
import pandas as pd
import warnings
import requests
import datetime as dt
warnings.filterwarnings('ignore')

from tqdm.notebook import  tqdm

with open('config.txt', 'r') as f:
    names = f.read().split('\n')

def process_roster(path):
    roster = pd.read_csv(path)
    roster['Game'] = roster.Game.str.split('@').apply(lambda x: set([y.strip() for y in x]))
    roster['Name'] = roster.Name
    roster['TeamAbbrev'] = roster.TeamAbbrev.apply(lambda x: set([x]))
    roster['OppAbbrev'] = roster.Game - roster.TeamAbbrev
    roster['Team'] = roster.TeamAbbrev.apply(lambda x: list(x)[0])
    roster['Opp'] = roster.OppAbbrev.apply(lambda x: list(x)[0])
    roster.Time = pd.to_datetime(roster.Time)
    roster.drop(['TeamAbbrev','OppAbbrev', 'Game'], axis = 1, inplace=True)
    return roster[['Name','Position', 'ID', 'Time', 'Salary', 'Team', 'Opp']]

def url_join(*args):
    return '/'.join(args)

class draftProcessor:
    def __init__(self, draft, load_ids = True):
        self.local_dir = 'data/NBA_API'
        self.base_url = "https://api-nba-v1.p.rapidapi.com"
        self.headers = {
            'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
            'x-rapidapi-key': "3c3966d785msh9c9c037d21ff8a9p1af469jsn475bf58bcbe1"
        }

        self.load_ids = load_ids
        self.draft = draft
        self.id_mapping = json.load(open(url_join(self.local_dir, 'id_mapping.json'), 'r'))

        if self.load_ids:
            self.get_player_ids()

        self.draft.index = [x.lower() for x in self.draft.index]
        self.draft = self.draft.join(pd.DataFrame.from_dict(self.id_mapping, orient='index').rename({0: 'api_id'}, axis = 1))
        self.draft.dropna(inplace=True)
        self.draft['api_id'] = self.draft['api_id'].astype(int)


    def get_player_ids(self):
        names = list(self.draft.index)

        for name in names:
            if name.lower() not in self.id_mapping:
                last_name = name.split(' ')[-1]
                print(f'id not found for {name}, getting id for {last_name}')
                try:
                    players = self.get_players({'name': last_name})
                except Exception as e:
                    print(f'failed. Error Message: {e}')
                    continue

                if type(players) == dict:
                    player_name = players['firstname'] + ' ' + players['lastname']
                    self.id_mapping[players[player_name.lower()]] = players['id']
                else:
                    for player in players:
                        player_name = player['firstname'] + ' ' + player['lastname']
                        player_id = player['id']
                        self.id_mapping[player_name.lower()] = player_id

                with open(url_join(self.local_dir, 'id_mapping.json'), 'w') as f:
                    json.dump(self.id_mapping,f)
            else:
                continue

    def get_players(self, params):
        url = url_join(self.base_url, 'players')
        print(url)
        resp= requests.get(url, headers = self.headers, params = params).json()
        return resp['response']


    def load_player_stats(self, params, ret = False):
        url = url_join(self.base_url, 'players','statistics')
        data_path = url_join(self.local_dir, 'stats', str(params['season']))
        params['id'] = int(params['id'])

        if f"{params['id']}.csv" in os.listdir(data_path):
            if ret:
                data = pd.read_csv(url_join(data_path, f"{params['id']}.csv"))
                data['id'] = params['id']
            else:
                return
        else:
            print(f"downloading data for id: {params['id']}")
            try:
                data = requests.get(url, headers=headers, params = params).json()
                data = pd.DataFrame(data['response']).drop(['team', 'player', 'game'], axis=1)
                data.to_csv(url_join(data_path, f"{params['id']}.csv"), index = False)
            except:
                print(f"failed. n_games = {len(data)}")
                return

        if ret:
            return data


In [12]:
from draft_kings import Client
from draft_kings import Sport
import pytz
def get_contests(names, date=dt.datetime.now()):
    contests = Client().contests(sport=Sport.NBA).contests
    names = [x.lower() for x in names]
    cinfo = {}
    for idx, contest in enumerate(contests):

        print(contest)
        if contest.name.lower() in names and contest.starts_at.replace(tzinfo=et) >= date.replace(tzinfo=et):
            cinfo[contest.name] = {
                'start_time': contest.starts_at,
                'id': contest.contest_id,
                'draft_id': contest.draft_group_id,
                'payout': contest.payout
            }
    return cinfo

et = pytz.timezone('America/New_York')
names = ['NBA SINGLE ENTRY $10 DOUBLE UP']
date = dt.datetime.now()

contests = get_contests(names, date)

ContestDetails(contest_id=133627093, draft_group_id=75319, entries_details=EntriesDetails(fee=5.0, maximum=237812, total=3367), fantasy_player_points=5.0, is_double_up=False, is_fifty_fifty=False, is_guaranteed=True, is_head_to_head=False, is_starred=True, name='NBA $1M Tip Off Special [$200K to 1st]', payout=1000000.0, sport=<Sport.NBA: 'NBA'>, starts_at=datetime.datetime(2022, 10, 18, 23, 30, tzinfo=datetime.timezone.utc))
ContestDetails(contest_id=133930292, draft_group_id=75812, entries_details=EntriesDetails(fee=15.0, maximum=4705, total=12), fantasy_player_points=15.0, is_double_up=False, is_fifty_fifty=False, is_guaranteed=True, is_head_to_head=False, is_starred=True, name='NBA $60K Fadeaway [$20K to 1st]', payout=60000.0, sport=<Sport.NBA: 'NBA'>, starts_at=datetime.datetime(2022, 10, 11, 23, 0, tzinfo=datetime.timezone.utc))
ContestDetails(contest_id=133930289, draft_group_id=75812, entries_details=EntriesDetails(fee=180.0, maximum=67, total=0), fantasy_player_points=180.0, is

In [31]:
from pydfs_lineup_optimizer import get_optimizer, Site, Sport
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASKETBALL)